<a href="https://colab.research.google.com/github/fishfishin/CT-denoise/blob/main/PTdqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import random
#import pydicom
import glob
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
#import visdom
import torchvision.transforms as T
import skimage.external.tifffile
import copy
import cv2
import math
from skimage import filters
from torch.autograd import Variable
from math import exp
from skimage.transform import resize
from skimage.transform import radon,iradon
from skimage.restoration import denoise_nl_means
#from google.colab import drive
"""
import tensorflow as tf
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
"""
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
print(dev)
#vis = visdom.Visdom()

def huber_loss_simple(a, b):
    error = a - b
    if abs(error) > 1.0:
        return abs(error) - 1/2
    return error*error / 2

class doubleDQN(nn.Module):

    def __init__(self, h, w,output):
        super(doubleDQN, self).__init__()

        # image patch size is   h x w
        #first two  shared conv layersoutput1,
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=(2),padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        #self.convs = nn.Conv2d(16, 32, kernel_size=3, stride=(2),padding=1)
        #self.bns = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=(2),padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        # patch size after conv layerconv2d_size_out( conv2d_size_out(
        def conv2d_size_out(size, kernel_size = 3, stride = 2):
            return size // stride  + 1
        convw =conv2d_size_out(w)
        convh = conv2d_size_out(h)

        
        
        linear_input_size = convw * convh  * 128
        #  Parameter tuning
        #  conv 128 Filter, kernel 3x3 
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1,padding=1)
        self.bn5 = nn.BatchNorm2d(128)
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, stride=1,padding=1)
        self.bn6 = nn.BatchNorm2d(128)
        #  FCL 256 neurons
        self.dense2 = nn.Linear(linear_input_size, 128)
        self.bn7 = nn.BatchNorm1d(128)
        self.head2 = nn.Linear(128, output)

        self.opt = optim.SGD(self.parameters(), lr=0.001, momentum=0.99)

    
    def forward(self, x):


        # 2 shared layers
        x = F.relu(self.bn1(self.conv1(x)))
        #x = F.relu(self.bns(self.convs(x)))
        #x = F.relu(self.bn2(self.conv2(x)))
         
        
        
        #  Parameter tuning
        out2 = F.relu(self.bn5(self.conv4(x)))
        out2 = F.relu(self.bn6(self.conv5(out2)))
        out2 = out2.view(out2.size(0), -1)
        out2 = F.relu(self.bn7(self.dense2(out2)))
        out2 = self.head2(out2)

        return  out2


    def update(self, x,target):
        #  inputs, labels = inputs.to(device), labels.to(device)
        x = x.cuda()
        out = self.forward(x)
        self.opt.zero_grad()
        
        loss1 = nn.SmoothL1Loss()
        upd_1 = loss1(out,target )
        
        upd_ =  upd_1 
        upd_.backward()
        
        self.opt.step()

        return upd_


####  1st path'patch size','searchsize',
Para = [ 'h']

####  2nd path
Actions = ['+1.5','1.1','null','0.9','-1.5']

MAX_EPOCHS = 50
DISCOUNT_RATE = 0.99
RESOLUTION = 256
PATCH_SIZE = [5,5]
Patch_num = RESOLUTION**2
PATCH_reward = 5           ### ssim and psnr
TARGET_UPDATE_STEP = 300
MAXSTEPS_FILTER=21   ####   21   if large then will get stuck in certain pattern
REPLAY_MEMORY = 4000000  ######### buffer？
BATCH_SIZE = 512

TUNING_STEP= 20
Nsample=2000
def replay_train(mainDQN: doubleDQN, targetDQN: doubleDQN, states, next_states, action, rewards) -> np.float64:

    X = torch.zeros(states.shape[0], 1, PATCH_SIZE[0],PATCH_SIZE[0], dtype=torch.float64)
    X1 = torch.zeros(states.shape[0], 1, PATCH_SIZE[0],PATCH_SIZE[0], dtype=torch.float64)
    X[:,0,:,:] = torch.from_numpy(states)
    X1[:,0,:,:] = torch.from_numpy(next_states)

    t = targetDQN(Variable(X1).cuda())  ## old version of NET
    y = mainDQN(Variable(X).cuda())
     
 
    ####  2nd path
    tem = torch.max(t, axis=1)
    temp = tem.values
    Q_target = rewards + temp.cpu().detach().numpy() *DISCOUNT_RATE
    for i in range(y.shape[0]):
        y[i,int(action[i])] = Q_target[i]

    

    return mainDQN.update(X,y)


def PSNR(img1, img2):
    D = np.array(img1 - img2, dtype=np.float64)
    D[:, :] = D[:, :]**2
    RMSE = D.sum()/img1.size
    psnr = 10*math.log10(float(1.0**2)/RMSE)
    return psnr


def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()

def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = Variable(_2D_window.expand(channel, 1, window_size, window_size).contiguous())
    return window

def _ssim(img1, img2, window, window_size, channel, size_average = True):
    mu1 = F.conv2d(img1, window, padding = window_size//2, groups = channel)
    mu2 = F.conv2d(img2, window, padding = window_size//2, groups = channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1*mu2

    sigma1_sq = F.conv2d(img1*img1, window, padding = window_size//2, groups = channel) - mu1_sq
    sigma2_sq = F.conv2d(img2*img2, window, padding = window_size//2, groups = channel) - mu2_sq
    sigma12 = F.conv2d(img1*img2, window, padding = window_size//2, groups = channel) - mu1_mu2

    C1 = 0.01**2
    C2 = 0.03**2

    ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)


def ssim(img1, img2, window_size = 11, size_average = True):
    (_, channel, _, _) = img1.size()
    #channel = 1
    window = create_window(window_size, channel)
    
    if img1.is_cuda:
        window = window.cuda(img1.get_device())
    window = window.type_as(img1)
    
    return _ssim(img1, img2, window, window_size, channel, size_average)

def divide_patch(fimg) -> np.float64:

    fimgpad = np.zeros((RESOLUTION + PATCH_SIZE[0] -1, RESOLUTION + PATCH_SIZE[0] -1),dtype =np.float64)
    fimgpad[int((PATCH_SIZE[0]+1)/2)-1:RESOLUTION+int((PATCH_SIZE[0]+1)/2)-1,int((PATCH_SIZE[0]+1)/2)-1:RESOLUTION+int((PATCH_SIZE[0]+1)/2)-1]=fimg
    state = np.zeros((Patch_num,PATCH_SIZE[0], PATCH_SIZE[0]))
    count = 0
    for xcord in range(RESOLUTION):
        for ycord in range(RESOLUTION):
            temp=fimgpad[xcord:xcord+PATCH_SIZE[0],ycord:ycord+PATCH_SIZE[0]]
            state[count,:,: ] = temp
            count += 1
    return state


def Denoise(state, action, parameter_value, GroundTruth, org_patch) -> np.float64:

    next_patch = np.zeros((Patch_num, PATCH_SIZE[0], PATCH_SIZE[0]),dtype=np.float64)
    #org_patch = _patch(original_image)
    #G = divide_patch(GroundTruth)
    # tuning parameter
    #d = torch.zeros((1,1,9,9),dtype=torch.float64)
    #current_image = np.reshape(state[:, int(PATCH_SIZE[0]//2), int(PATCH_SIZE[0]//2)], (RESOLUTION, RESOLUTION), order='A')
    # reward = np.zeros(Patch_num)
    """
    S = torch.zeros(1, 1,RESOLUTION,RESOLUTION,dtype=torch.float64)
    S[:,:,:,:] = torch.from_numpy(current_image)
    rs = Q.forward(S)
    """
    for idx in range(Patch_num):
        #temp =S
        if 0==0: 
            if action[idx]==0:
                parameter_value[idx,0] *=  1.5
            if action[idx]==1:
                parameter_value[idx,0] *= 1.1
            if action[idx]==3:
                parameter_value[idx,0] *= 0.9 
            if action[idx]==4:
                parameter_value[idx,0] *=  0.5 
        
                
        next_patch[idx, :, :] = denoise_nl_means(org_patch[idx,:,:],h= parameter_value[idx,0],fast_mode=True,
                                            patch_size=2,patch_distance=1,multichannel=False)  
        #reward[idx] = 1.0 / (np.sqrt(np.sum((G[idx, int(PATCH_SIZE[0]//2), int(PATCH_SIZE[0]//2 ] - final_patch[int(PATCH_SIZE[0]//2), int(PATCH_SIZE[0]//2])**2)) +0.00001) 
        # -1.0 / (np.sqrt(np.sum(G[idx, int(PATCH_SIZE[0]//2), int(PATCH_SIZE[0]//2] - state[idx, int(PATCH_SIZE[0]//2), int(PATCH_SIZE[0]//2])**2))+0.00001)
        """
            a = idx/RESOLUTION
            b = idx%RESOLUTION
            #temp[:, :,a:a+PATCH_SIZE[0], b:b+PATCH_SIZE[0]] = torch.from_numpy(final_patch)  
            temp[:, :,a, b]=  final_patch[int(PATCH_SIZE[0]//2),int(PATCH_SIZE[0]//2)]   
            reward[idx] = Q.forward(temp) - rs
        """
        
        #reward[idx] = -np.sqrt(np.sum((G[idx, :, :] - final_patch)**2)) + np.sqrt(np.sum((G[idx, :, :] - state[idx, :, :])**2))
        #print("patch: {}".format(idx))
    
    ############# NLM
    next_state = next_patch
    ### how to stitch  256x 256 patches into a single image fimg  after filtering
    next_img = np.reshape(next_patch[:, int(PATCH_SIZE[0]//2), int(PATCH_SIZE[0]//2)], (RESOLUTION, RESOLUTION), order='A')
    current_image = np.reshape(state[:, int(PATCH_SIZE[0]//2), int(PATCH_SIZE[0]//2)], (RESOLUTION, RESOLUTION), order='A')
    
    dist1img = current_image - GroundTruth
    dist2img = next_img - GroundTruth  
    dist2 = np.reshape(dist2img, (Patch_num), order='A')
    dist1imgLarge = np.zeros((RESOLUTION+PATCH_reward-1,RESOLUTION+PATCH_reward-1))
    margin = int((PATCH_reward-1)/2)
    dist1imgLarge[margin:RESOLUTION+margin,margin:RESOLUTION+margin]=np.absolute(dist1img)
    dist2imgLarge = np.zeros((RESOLUTION + PATCH_reward-1, RESOLUTION + PATCH_reward-1))
    dist2imgLarge[margin:RESOLUTION + margin, margin:RESOLUTION + margin] = np.absolute(dist2img)
    rewardimg = np.zeros((RESOLUTION,RESOLUTION))
    
    for i in range(RESOLUTION):
        for j in range(RESOLUTION):

            rewardimg[i,j]= 1/(np.sum(dist2imgLarge[i:i+PATCH_reward,j:j+PATCH_reward])+0.00001) - 1/(np.sum(dist1imgLarge[i:i+PATCH_reward,j:j+PATCH_reward])+0.00001)
    reward = np.reshape(rewardimg,(Patch_num),order='A')

            
    
    return next_state, reward, parameter_value, next_img


def main():


    #drive.mount('/content/drive')
    """
    path = "drive/My Drive/Colab Notebooks/LDCT/L004/Full dose images/*.dcm"
    TrueImgTrain = np.zeros((97,256,256))
    i =0
    for f in glob.glob(path):
      ds = pydicom.dcmread(f)
      data = resize(ds.pixel_array/4095.0,(256,256),anti_aliasing=False)
      TrueImgTrain[i,:,:] =data    
      i+=1
    path = "drive/My Drive/Colab Notebooks/LDCT/L004/Low Dose Images/*.dcm"
    train_data = np.zeros((97,256,256))
    i =0
    for f in glob.glob(path):
      ds = pydicom.dcmread(f)
      data = resize(ds.pixel_array/4095.0,(256,256),anti_aliasing=False)
      train_data[i,:,:] =data
      i+=1
    
    np.save("drive/My Drive/Colab Notebooks/LDCT/L004/full",TrueImgTrain)
    np.save("drive/My Drive/Colab Notebooks/LDCT/L004/low",train_data)
    """
    TrueImgTrain =np.load( "C:/Users/ZhenjuYin/Downloads/LDCT-and-Projection-data/C016/full.npy",allow_pickle=True)
    train_data= np.load("C:/Users/ZhenjuYin/Downloads/LDCT-and-Projection-data/C016/low.npy",allow_pickle=True)
    TrueImgTrain = TrueImgTrain[0:1,:,:]
    train_data = train_data[0:1,:,:]
    datasize = TrueImgTrain.shape
    
    plt.set_cmap(plt.gray())

    # (None, channel, H, w, depth) for Volume
    mainDQN = doubleDQN(PATCH_SIZE[0], PATCH_SIZE[1], len(Actions)).cuda().double()
    targetDQN = doubleDQN(PATCH_SIZE[0], PATCH_SIZE[1],len(Actions)).cuda().double()

    state_sel = np.zeros((REPLAY_MEMORY, PATCH_SIZE[0], PATCH_SIZE[0]))
    next_state_sel = np.zeros((REPLAY_MEMORY, PATCH_SIZE[0], PATCH_SIZE[0]))
    action_sel = np.zeros((REPLAY_MEMORY))
    reward_sel = np.zeros((REPLAY_MEMORY))
  
    
    indicator = 0 
    if MAX_EPOCHS>0:  ########################## len(Para),len(Para),
                   
            ##  slabs  x  256**2 x 9 x 9
            
            ############ State_initial shouldnt be changed !!!!!
            State_initial = np.zeros((datasize[0], Patch_num, PATCH_SIZE[0],PATCH_SIZE[0])) 
            org = np.zeros((datasize[0], Patch_num, PATCH_SIZE[0],PATCH_SIZE[0]))
            parameter_value = 0.5*np.ones((Patch_num,len(Para)))
        
            #parameter_value = np.load("drive/My Drive/Colab Notebooks/LDCT/C016/parameter.npy",allow_pickle=True)
            # for each image initializer
            for IMG in range(datasize[0]):
                state = divide_patch(  train_data[ IMG,:,:]  )  ##  256**2 x 9 x 9              
                GroundTruth = TrueImgTrain[IMG,:, : ] 
                org[ IMG,:,:,:] = state
                    
                ## initialize the 1st and 2nd paths 
                
                action = 2 * np.ones((Patch_num))
                next_state, reward, parameter_value, img = Denoise( state,  action, parameter_value , GroundTruth, org[ IMG,:,:,:] )
                #skimage.external.tifffile.imshow(img)                
                State_initial[ IMG, :, :, :] = next_state
                #print(IMG)

            count_memory = 0

            for episode in range(MAX_EPOCHS-1):
                State = np.zeros((datasize[0], Patch_num, PATCH_SIZE[0],PATCH_SIZE[0])) 
                e = 0.999 / ((episode / 50) + 1)
                if e<0.1:
                    e=0.1
                step_count = 0
                State[:,:,:,:]  = State_initial[:,:,:,:] 
                parameter_value = 0.5*np.ones((Patch_num,len(Para)))
                #print(State[0,25700,3,3])
                for ITER_NUM in range(MAXSTEPS_FILTER):  
                  
                    for IMG_IDX in range(datasize[0]):
                       
                        state = State[ IMG_IDX, :, :, :]     ##  slabs x num of patches x 9 x 9
                        GroundTruth = TrueImgTrain[IMG_IDX,:, : ]                      
                        action = 2 * np.ones((Patch_num))

                        # random select patches and     action  for each image
                        flag = np.random.rand(Patch_num)
                        count_patch = 0
                        length_patch = 0
                        for idx in range(Patch_num):
                            if flag[idx]>=e:
                                length_patch += 1
                        
                        # yy  : patch samples
                        print("start sampling patches")
                        yy = torch.zeros(length_patch, 1, PATCH_SIZE[0],PATCH_SIZE[0], dtype=torch.float64)
                        for idx in range(Patch_num):
                            if flag[idx]<e:
                                action[idx] = np.random.randint(len(Actions), size=1)
                               
                            if flag[idx]>=e:
                                yy[count_patch,0,:, :] = torch.from_numpy(state[idx,:,:])
                                count_patch += 1
                        #print(yy.shape)
                        y = mainDQN(Variable(yy).cuda())
                       
                        action_yy = torch.argmax(y, axis=1)
                     
                        #### action and paramter chosen
                        count_patch=0
                        for idx in range(Patch_num):
                            if flag[idx] >= e:
                                action[idx] = action_yy[count_patch]
                               
                                count_patch += 1
                        print("strat denoise")
                        next_state, reward, parameter_value, img = Denoise( state,   action,parameter_value, GroundTruth, org[ IMG_IDX,:,:,:] )
                        psnr =PSNR(GroundTruth, img )
                        
                        img1 = torch.zeros(1, 1, datasize[1],datasize[2],dtype=torch.float64)
                        img2 = torch.zeros(1, 1, datasize[1],datasize[2],dtype=torch.float64)
                        img1[:,0,:,:] = torch.from_numpy(GroundTruth)
                        img2[:,0,:,:] = torch.from_numpy(img)
                        ssim_ = ssim(img1,img2)
                        
                        if  ITER_NUM %10==0 :
                          fig, axs = plt.subplots(1, 1,figsize=(10,10))
                          fig.suptitle(" iteration: {}".format(ITER_NUM ))
                          axs.imshow(img, cmap='gray')
                          plt.show()
                        
                        print(" current PSNR : {} ssim : {} reward:{} ".format(psnr,ssim_,np.mean(reward)))
                        
                        ###################  ? random replacement
                        sel_prob = 0.01
                        flag1 = np.random.rand(Patch_num)
                        flag2 = np.zeros([Patch_num])
                        for idx in range(Patch_num):
                            if flag1[idx]>=sel_prob:
                                flag2[idx] = 0
                            if flag1[idx]<sel_prob:  ##### chosen
                                flag2[idx] = 1

                        sel_num = int(np.sum(flag2))

                        ##### refresh the buffer randomly
                        if count_memory+sel_num<=REPLAY_MEMORY-2:
                            for idx in range(Patch_num):
                                if flag1[idx]<sel_prob:
                                    state_sel[count_memory,:,:] = state[idx,:,:]
                                    next_state_sel[count_memory,:,:] = next_state[idx,:,:]
                                    action_sel[count_memory]=action[idx]
                                    reward_sel[count_memory] = reward[idx]
                                    
                                    count_memory += 1
                        else:
                            indicator = 1
                            for idx in range(Patch_num):
                                if flag1[idx]<sel_prob:
                                    state_sel[count_memory,:] = state[idx,:,:]
                                    next_state_sel[count_memory,:] = next_state[idx,:,:]
                                    action_sel[count_memory]=action[idx]
                                    reward_sel[count_memory] = reward[idx]
                                    
                                    if count_memory == REPLAY_MEMORY - 1:
                                        count_memory = 0
                                        print('Replay Memory is full')
                                    else:
                                        count_memory += 1
                        if indicator == 0:
                            replay_size = count_memory +  1
                        else:
                            replay_size = REPLAY_MEMORY

                        if replay_size > BATCH_SIZE:

                            #print(" tuning")
                            
                            for i in range(TUNING_STEP):
                                shuffle_order = np.arange(replay_size)
                                np.random.shuffle(shuffle_order)
                                minibatch_state = state_sel[shuffle_order[0:BATCH_SIZE], :, :]
                                minibatch_next_state = next_state_sel[shuffle_order[0:BATCH_SIZE],:,:]
                                minibatch_action = action_sel[shuffle_order[0:BATCH_SIZE]]
                                minibatch_reward = reward_sel[shuffle_order[0:BATCH_SIZE]]
                                
                                loss = replay_train(mainDQN, targetDQN, minibatch_state,minibatch_next_state,minibatch_action, minibatch_reward)
                                if step_count % TARGET_UPDATE_STEP == 0:
                                    targetDQN = copy.deepcopy(mainDQN)
                                step_count += 1

                        ###################   randomly store patches
                        """
                        shuffle_order = np.arange(Patch_num)
                        np.random.shuffle(shuffle_order)

                        ##### refresh the buffer randomly
                        if count_memory+Nsample<=REPLAY_MEMORY-2:
                                             
                          state_sel[count_memory:count_memory+Nsample,:,:] = state[shuffle_order[0:Nsample],:,:]
                          next_state_sel[count_memory:count_memory+Nsample,:,:] = next_state[shuffle_order[0:Nsample],:,:]
                          action_sel[count_memory:count_memory+Nsample]=action[shuffle_order[0:Nsample]]
                         
                          reward_sel[count_memory:count_memory+Nsample] = reward[shuffle_order[0:Nsample]]
                    
                          count_memory += Nsample
                        else:
                          indicator = 1
                          remain = REPLAY_MEMORY- count_memory
                          state_sel[count_memory:count_memory+remain,:,:] = state[shuffle_order[0:remain],:,:]
                          next_state_sel[count_memory:count_memory+remain,:,:] = next_state[shuffle_order[0:remain],:,:]
                          action_sel[count_memory:count_memory+remain]=action[shuffle_order[0:remain]]
                          
                          reward_sel[count_memory:count_memory+remain] = reward[shuffle_order[0:remain]]
                          print('Replay Memory is full')
                          count_memory = 0
                          remain_ = Nsample- remain
                          state_sel[count_memory:count_memory+remain_,:,:] = state[shuffle_order[remain:Nsample],:,:]
                          next_state_sel[count_memory:count_memory+remain_,:,:] = next_state[shuffle_order[remain:Nsample],:,:]
                          action_sel[count_memory:count_memory+remain_]=action[shuffle_order[remain:Nsample]]
                          
                          reward_sel[count_memory:count_memory+remain_] = reward[shuffle_order[remain:Nsample]]
                        if indicator == 0:
                            replay_size = count_memory +  1
                        else:
                            replay_size = REPLAY_MEMORY

                        if replay_size > BATCH_SIZE:

                            print(" tuning")
                            
                            for i in range(TUNING_STEP):
                                shuffle_order = np.arange(replay_size)
                                np.random.shuffle(shuffle_order)
                                minibatch_state = state_sel[shuffle_order[0:BATCH_SIZE], :, :]
                                minibatch_next_state = next_state_sel[shuffle_order[0:BATCH_SIZE],:,:]
                                minibatch_action = action_sel[shuffle_order[0:BATCH_SIZE]]
                               
                                minibatch_reward = reward_sel[shuffle_order[0:BATCH_SIZE]]
                                
                                loss = replay_train(mainDQN, targetDQN, minibatch_state,minibatch_next_state,minibatch_action,minibatch_reward)
                                if step_count % TARGET_UPDATE_STEP == 0:
                                    targetDQN = copy.deepcopy(mainDQN)
                                step_count += 1
                                #print(step_count)
                        """
                        State[IMG_IDX, :, :, :] = next_state
                        #np.save("drive/My Drive/Colab Notebooks/LDCT/C016/parameter",parameter_value)
                        

                    print("Episode: {}  Iteration: {} Loss: {}".format(episode, ITER_NUM, loss))

                CHECK = episode+1
    # save the trained networks
    PATH = "C:/Users/ZhenjuYin/Downloads/LDCT-and-Projection-data/mainDQN"
    torch.save(mainDQN.state_dict(), PATH)
    PATH = "C:/Users/ZhenjuYin/Downloads/LDCT-and-Projection-data//targetDQN"
    torch.save(targetDQN.state_dict(), PATH)
    PATH = "C:/Users/ZhenjuYin/Downloads/LDCT-and-Projection-data/parameter_value"
    np.save(PATH,parameter_value)

if __name__ == '__main__':
    main()